In [2]:
import heapq
import csv
import re
import time
import cProfile
from collections import defaultdict
import networkx as nx
import itertools
import pandas as pd

def neighborhood_lower_bound(graph):
    nodes = list(graph.nodes())
    n = len(nodes)
    lower_bounds = {}

    k = 2
    max_iterations = 100000  # Set maximum iterations

    Y = {}
    S_un = {}
    nVisited = {}
    finished = {}

    for s in nodes:
        degree_s = graph.degree(s)
        Y[(1, s)] = degree_s
        S_un[s] = degree_s
        nVisited[s] = degree_s + 1
        finished[s] = False

    nFinished = 0

    while nFinished < n and k <= max_iterations:
        for s in nodes:
            if finished[s]:
                continue
            if k == 2:
                Y[(k, s)] = sum(Y.get((k-1, w), 0) for w in graph.neighbors(s)) - graph.degree(s)
            else:
                Y[(k, s)] = sum(Y.get((k-1, w), 0) for w in graph.neighbors(s)) - Y.get((k-2, s), 0) * (graph.degree(s) - 1)
        
        for s in nodes:
            if finished[s]:
                continue
            y_k_minus_2 = Y.get((k-2, s), 0)
            y_k_minus_1 = Y.get((k-1, s), 0)
            nVisited[s] += y_k_minus_1
            
            if nVisited[s] < n:
                S_un[s] += k * y_k_minus_1
            else:
                S_un[s] += k *(n - (nVisited[s] - y_k_minus_1))
                nFinished += 1
                finished[s] = True
            Y[(k-2, s)] = y_k_minus_1
            Y[(k-1, s)] = Y[(k, s)]
        
        k += 1

    for v in nodes:
        lower_bounds[v] = S_un[v] / (n - 1)

    return lower_bounds

def updateBoundsBFSCut(v, graph, x):
    """
    Calculate the lower bound of the cut size that separates vertex v.

    Parameters:
        v: Starting vertex.
        graph: networkx DiGraph. Nodes must have 'r' attribute.
        Farn: Dictionary that stores the lower bounds (must be initialized).
        Top: List of top k nodes (must be initialized).
        x: A threshold.

    Returns:
        If the cut value exceeds x, return +∞; otherwise, return the calculated cut value or the current Farn[v] if unchanged.
    """
    
    nodes = list(graph.nodes())
    n = len(nodes)
    Q = [(0, v)]  # Priority queue to track BFS (distance, node)
    heapq.heapify(Q)
    visited = {v}
    d = 0
    S = 0
    y = graph.degree(v) - 1
    nd = 1

    while Q:
        dist, u = heapq.heappop(Q)
        if dist > d:
            d += 1
            # LCUT calculation
            LCUT = ((d + 2) * (n - nd) + S - y) / (n - 1)

            if LCUT >= x:
                return float('inf')
            y = 0  # Reset y

        for w in graph.neighbors(u):
            if w not in visited:
                visited.add(w)
                dist_vw = nx.shortest_path_length(graph, v, w)  # Assume a shortest path exists
                heapq.heappush(Q, (dist_vw, w))
                S += dist_vw  # Distance d(v, w)
                y += graph.degree(w)
                nd += 1
            else:
                LCUT += 1 / (n - 1)

    # Final calculation
    LCUT_final = S / (n - 1)

    return LCUT_final

def create_mention_graph_with_centrality(filepath, top_k=10):
    start_time = time.time()

    data = pd.read_csv(filepath, sep="\t", header=None)
    directed_graph = nx.DiGraph()
    directed_graph.add_edges_from(data.values.tolist())
    graph = directed_graph.to_undirected()

    print(f"Undirected graph created with {graph.number_of_nodes()} nodes and {graph.number_of_edges()} edges.")
    print("Calculating initial lower bounds...")

    initial_lower_bounds = neighborhood_lower_bound(graph)
        k_lowest_farness = heapq.nsmallest(top_k, initial_lower_bounds.items(), key=lambda item: item[1])
    print(f"\n{top_k} nodes with lowest farness values:")
    for node, farness in k_lowest_farness:
        print(f"Node: {node}, Farness: {farness}")
    
    Farn = initial_lower_bounds.copy()
    Top = []
    Q = [(Farn[node], node) for node in graph.nodes()]
    heapq.heapify(Q)

    initial_threshold = float('inf')

    _, v = heapq.heappop(Q)
    
    print(f"Processing node {v}")

    threshold = initial_threshold if len(Top) < top_k else Farn[Top[-1]]
    refined_bound = updateBoundsBFSCut(v, graph, threshold)
    
    print(f"Refined bound for node {v}: {refined_bound}")

    Farn[v] = refined_bound

    while Q:
        if len(Top) < top_k:
            Top.append(v)
            Top.sort(key=lambda node: Farn[node])
            print(f"Added node {v} to Top. Current Top: {Top}")
        elif refined_bound < Farn[Top[-1]]:
            Top.append(v)
            Top.sort(key=lambda node: Farn[node])
            Top = Top[:top_k]  # Maintain Top's length to top_k
            print(f"Added node {v} to Top and trimmed. Current Top: {Top}")
        else:
            print(f"Node {v} not added to Top as its bound is not better than current top")

        _, v = heapq.heappop(Q)
    
        print(f"Processing node {v}")

        threshold = initial_threshold if len(Top) < top_k else Farn[Top[-1]]
        refined_bound = updateBoundsBFSCut(v, graph, threshold)
    
        print(f"Refined bound for node {v}: {refined_bound}")

        Farn[v] = refined_bound
        
        if len(Top) == top_k and Farn[v] >= Farn[Top[-1]]:
            print(Farn[Q[0][1]])
            print(Farn[Top[-1]])
            print("Remaining nodes in Q cannot improve Top. Stopping.")
            break

    print("\nFinal Top 10 nodes by refined lower centrality bound:")
    for i, node in enumerate(Top, 1):
        print(f"{i}. Node: {node}, Refined Lower Centrality: {Farn[node]:.10f}")

    end_time = time.time()
    total_time = end_time - start_time
    print(f"\nTotal time taken: {total_time:.4f} seconds")

    return graph

if __name__ == "__main__":
    filepath = "com-youtube.ungraph.tsv"  # Replace with your file path
    print("Creating graph...")
    graph_with_centrality = create_mention_graph_with_centrality(filepath)

In [ ]:
import heapq
import csv
import re
import time
import cProfile
from collections import defaultdict
import networkx as nx
import itertools
import pandas as pd

def calculate_closeness_centrality(graph, top_k=10):
    """
    Calculates the closeness centrality of nodes in a graph and returns the top k nodes with the highest centrality.

    Args:
        graph (networkx.Graph): The input graph.
        top_k (int): The number of top nodes to return.

    Returns:
        dict: A dictionary containing the top k nodes and their closeness centrality values.
    """
    # Calculate closeness centrality for each node
    closeness_centrality = nx.closeness_centrality(graph)

    # Get the top k nodes with the highest closeness centrality
    top_nodes = heapq.nlargest(top_k, closeness_centrality, key=closeness_centrality.get)
    top_centrality = {node: closeness_centrality[node] for node in top_nodes}

    return top_centrality

def create_mention_graph_with_centrality(filepath, top_k=10):
    start_time = time.time()

    data = pd.read_csv(filepath, sep="\t", header=None)
    directed_graph = nx.DiGraph()
    directed_graph.add_edges_from(data.values.tolist())
    graph = directed_graph.to_undirected()

    print(f"Undirected graph created with {graph.number_of_nodes()} nodes and {graph.number_of_edges()} edges.")

    print("Calculating top k nodes by closeness centrality...")
    top_closeness = calculate_closeness_centrality(graph, top_k)

    print("\nTop 10 nodes by closeness centrality:")
    for i, (node, centrality) in enumerate(top_closeness.items(), 1):
        print(f"{i}. Node: {node}, Closeness Centrality: {centrality:.10f}")

    end_time = time.time()
    total_time = end_time - start_time
    print(f"\nTotal time taken: {total_time:.4f} seconds")

    return graph

if __name__ == "__main__":
    filepath = "com-youtube.ungraph.tsv"  # Replace with your file path
    print("Creating graph...")
    graph_with_centrality = create_mention_graph_with_centrality(filepath)

In [ ]:
import heapq
import csv
import re
import time
import cProfile
from collections import defaultdict
import networkx as nx
import itertools
import pandas as pd

def pagerank_lower_bound(graph):
    # Calculate PageRank for each node
    page_ranks = nx.pagerank(graph)
    
    # Use the reciprocal of PageRank as a lower bound
    lower_bounds = {node: 1 / page_ranks[node] for node in graph.nodes()}
    
    return lower_bounds

def updateBoundsBFSCut(v, graph, x):
    """
    Calculates a lower bound on the size of the cut separating vertex v.

    Args:
        v: The starting vertex.
        graph: A networkx DiGraph. Nodes must have an 'r' attribute.
        x: A threshold value.

    Returns:
        +∞ if the cut value exceeds x; otherwise, returns the calculated cut value, 
        or the current Farn[v] if no change.
    """
    
    nodes = list(graph.nodes())
    n = len(nodes)
    Q = [(0,v)] # Priority queue for BFS (distance, node)
    heapq.heapify(Q)
    visited = {v}
    d = 0
    S = 0
    y = graph.degree(v) - 1
    nd = 1

    while Q:
        dist, u = heapq.heappop(Q)
        if dist > d:
            d += 1
            #LCUT calculation
            LCUT = ((d+2)*(n-nd) + S - y )/(n-1)

            if LCUT >= x:
                return float('inf')
            y = 0  # Reset y

        for w in graph.neighbors(u):
            if w not in visited:
                visited.add(w)
                dist_vw = nx.shortest_path_length(graph,v,w) # Assumes shortest path exists
                heapq.heappush(Q,(dist_vw,w))
                S += dist_vw  # Distance d(v,w)
                y += graph.degree(w)
                nd += 1
            else:
                LCUT = LCUT + 1/(n-1)

    #Final calculation
    LCUT_final = S / (n-1)

    return LCUT_final


def create_mention_graph_with_centrality(filepath, top_k=10):
    start_time = time.time()

    data = pd.read_csv(filepath, sep="\t", header=None)
    directed_graph = nx.DiGraph()
    directed_graph.add_edges_from(data.values.tolist())
    graph = directed_graph.to_undirected()

    print(f"Undirected graph created with {graph.number_of_nodes()} nodes and {graph.number_of_edges()} edges.")
    print("Calculating initial lower bounds using PageRank...")

    initial_lower_bounds = pagerank_lower_bound(graph)
    
    k_lowest_farness = heapq.nsmallest(top_k, initial_lower_bounds.items(), key=lambda item: item[1])
    print(f"\n{top_k} nodes with lowest initial farness values:")
    for node, farness in k_lowest_farness:
        print(f"Node: {node}, Farness: {farness}")
    
    Farn = initial_lower_bounds.copy()
    Top = []
    Q = [(Farn[node], node) for node in graph.nodes()]
    heapq.heapify(Q)

    initial_threshold = float('inf')


    _, v = heapq.heappop(Q)
    
    print(f"Processing node {v}")

    threshold = initial_threshold if len(Top) < top_k else Farn[Top[-1]]
    refined_bound = updateBoundsBFSCut(v, graph, threshold)
    
    print(f"Refined bound for node {v}: {refined_bound}")

    Farn[v] = refined_bound

    while Q:
        
        if len(Top) < top_k:
            Top.append(v)
            Top.sort(key=lambda node: Farn[node])
            print(f"Added node {v} to Top. Current Top: {Top}")
        elif refined_bound < Farn[Top[-1]]:
            Top.append(v)
            Top.sort(key=lambda node: Farn[node])
            Top = Top[:top_k]  # Keep Top length at top_k
            print(f"Added node {v} to Top and trimmed. Current Top: {Top}")
        else:
            print(f"Node {v} not added to Top as its bound is not better than current top")

        _, v = heapq.heappop(Q)
    
        print(f"Processing node {v}")

        threshold = initial_threshold if len(Top) < top_k else Farn[Top[-1]]
        refined_bound = updateBoundsBFSCut(v, graph, threshold)
    
        print(f"Refined bound for node {v}: {refined_bound}")

        Farn[v] = refined_bound
        
        if len(Top) == top_k and Farn[v] >= Farn[Top[-1]]:
            print(Farn[v])
            print(Farn[Top[-1]])
            print("Remaining nodes in Q cannot improve Top. Stopping.")
            break

    print("\nFinal Top 10 nodes by refined lower centrality bound:")
    for i, node in enumerate(Top, 1):
        print(f"{i}. Node: {node}, Refined Lower Centrality: {Farn[node]:.10f}")

    end_time = time.time()
    total_time = end_time - start_time
    print(f"\nTotal time taken: {total_time:.4f} seconds")

    return graph

if __name__ == "__main__":
    filepath = "com-youtube.ungraph.tsv"  # Replace with your file path
    print("Creating graph...")
    graph_with_centrality = create_mention_graph_with_centrality(filepath)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import heapq
import csv
import re
import time
import cProfile
from collections import defaultdict
import networkx as nx
import itertools
import pandas as pd


def neighborhood_lower_bound(graph):
    nodes = list(graph.nodes())
    n = len(nodes)
    lower_bounds = {}

    k = 2
    max_iterations = 100000  # Set maximum iterations

    Y = {}
    S_un = {}
    nVisited = {}
    finished = {}

    for s in nodes:
        degree_s = graph.degree(s)
        Y[(1, s)] = degree_s
        S_un[s] = degree_s
        nVisited[s] = degree_s + 1
        finished[s] = False

    nFinished = 0

    while nFinished < n and k <= max_iterations:
        for s in nodes:
            if finished[s]:
                continue
            if k == 2:
                Y[(k, s)] = sum(Y.get((k - 1, w), 0) for w in graph.neighbors(s)) - graph.degree(s)
            else:
                Y[(k, s)] = sum(Y.get((k - 1, w), 0) for w in graph.neighbors(s)) - Y.get((k - 2, s), 0) * (
                            graph.degree(s) - 1)

        for s in nodes:
            if finished[s]:
                continue
            y_k_minus_2 = Y.get((k - 2, s), 0)
            y_k_minus_1 = Y.get((k - 1, s), 0)
            nVisited[s] += y_k_minus_1

            if nVisited[s] < n:
                S_un[s] += k * y_k_minus_1
            else:
                S_un[s] += k * (n - (nVisited[s] - y_k_minus_1))
                nFinished += 1
                finished[s] = True
            Y[(k - 2, s)] = y_k_minus_1
            Y[(k - 1, s)] = Y[(k, s)]

        k += 1

    for v in nodes:
        lower_bounds[v] = S_un[v] / (n - 1)

    return lower_bounds


def updateBoundsBFSCut(v, graph, x):
    """
    Calculate the lower bound of the cut size that separates vertex v.

    Parameters:
        v: Starting vertex.
        graph: networkx DiGraph. Nodes must have 'r' attribute.
        Farn: Dictionary that stores the lower bounds (must be initialized).
        Top: List of top k nodes (must be initialized).
        x: A threshold.

    Returns:
        If the cut value exceeds x, return +∞; otherwise, return the calculated cut value or the current Farn[v] if unchanged.
    """

    nodes = list(graph.nodes())
    n = len(nodes)
    Q = [(0, v)]  # Priority queue to track BFS (distance, node)
    heapq.heapify(Q)
    visited = {v}
    d = 0
    S = 0
    y = graph.degree(v) - 1
    nd = 1

    while Q:
        dist, u = heapq.heappop(Q)
        if dist > d:
            d += 1
            # LCUT calculation
            LCUT = ((d + 2) * (n - nd) + S - y) / (n - 1)

            if LCUT >= x:
                return float('inf')
            y = 0  # Reset y

        for w in graph.neighbors(u):
            if w not in visited:
                visited.add(w)
                dist_vw = nx.shortest_path_length(graph, v, w)  # Assume a shortest path exists
                heapq.heappush(Q, (dist_vw, w))
                S += dist_vw  # Distance d(v, w)
                y += graph.degree(w)
                nd += 1
            else:
                LCUT += 1 / (n - 1)

    # Final calculation
    LCUT_final = S / (n - 1)

    return LCUT_final


def pagerank_lower_bound(graph):
    # Calculate PageRank for each node
    page_ranks = nx.pagerank(graph)

    # Use the reciprocal of PageRank as a lower bound
    lower_bounds = {node: 1 / page_ranks[node] for node in graph.nodes()}

    return lower_bounds


def calculate_top_k_closeness_centrality_nx(graph, top_k=10):
    start_time = time.time()

    print("Calculating closeness centrality using NetworkX...")
    closeness_centrality = nx.closeness_centrality(graph)

    top_k_nodes = heapq.nlargest(top_k, closeness_centrality.items(), key=lambda x: x[1])

    print(f"\nTop {top_k} nodes by closeness centrality (NetworkX):")
    for i, (node, centrality) in enumerate(top_k_nodes, 1):
        print(f"{i}. Node: {node}, Closeness Centrality: {centrality:.10f}")

    end_time = time.time()
    total_time = end_time - start_time
    print(f"\nTotal time taken for NetworkX calculation: {total_time:.4f} seconds")

    return [node for node, _ in top_k_nodes]


def calculate_top_k_closeness_centrality_paper(graph, top_k=10):
    start_time = time.time()

    print(f"Processing graph with {graph.number_of_nodes()} nodes and {graph.number_of_edges()} edges.")
    print("Calculating initial lower bounds...")

    initial_lower_bounds = neighborhood_lower_bound(graph)
    k_lowest_farness = heapq.nsmallest(top_k, initial_lower_bounds.items(), key=lambda item: item[1])
    # print(f"\n{top_k} nodes with lowest farness values:")
    # for node, farness in k_lowest_farness:
    # print(f"Node: {node}, Farness: {farness}")

    Farn = initial_lower_bounds.copy()
    Top = []
    Q = [(Farn[node], node) for node in graph.nodes()]
    heapq.heapify(Q)

    initial_threshold = float('inf')

    _, v = heapq.heappop(Q)

    # print(f"Processing node {v}")

    threshold = initial_threshold if len(Top) < top_k else Farn[Top[-1]]
    refined_bound = updateBoundsBFSCut(v, graph, threshold)

    # print(f"Refined bound for node {v}: {refined_bound}")

    Farn[v] = refined_bound

    while Q:
        if len(Top) < top_k:
            Top.append(v)
            Top.sort(key=lambda node: Farn[node])
            # print(f"Added node {v} to Top. Current Top: {Top}")
        elif refined_bound < Farn[Top[-1]]:
            Top.append(v)
            Top.sort(key=lambda node: Farn[node])
            Top = Top[:top_k]  # Maintain Top's length to top_k
            # print(f"Added node {v} to Top and trimmed. Current Top: {Top}")
        # else:
        # print(f"Node {v} not added to Top as its bound is not better than current top")

        _, v = heapq.heappop(Q)

        # print(f"Processing node {v}")

        threshold = initial_threshold if len(Top) < top_k else Farn[Top[-1]]
        refined_bound = updateBoundsBFSCut(v, graph, threshold)

        # print(f"Refined bound for node {v}: {refined_bound}")

        Farn[v] = refined_bound

        # if len(Top) == top_k and Farn[v] >= Farn[Top[-1]]:
        # print(Farn[Q[0][1]])
        # print(Farn[Top[-1]])
        # print("Remaining nodes in Q cannot improve Top. Stopping.")
        # break

    # print("\nFinal Top 10 nodes by refined closeness centrality:")
    for i, node in enumerate(Top, 1):
        closeness = 1 / Farn[node] if Farn[node] != 0 else float('inf')
        print(f"{i}. Node: {node}, Refined Closeness Centrality paper: {closeness:.10f}")

    end_time = time.time()
    total_time = end_time - start_time
    print(f"\nTotal time taken: {total_time:.4f} seconds")

    return Top


def calculate_top_k_closeness_centrality_pagerank(graph, top_k=10):
    start_time = time.time()

    print(f"Processing graph with {graph.number_of_nodes()} nodes and {graph.number_of_edges()} edges.")
    print("Calculating initial lower bounds...")

    initial_lower_bounds = pagerank_lower_bound(graph)
    k_lowest_farness = heapq.nsmallest(top_k, initial_lower_bounds.items(), key=lambda item: item[1])
    # print(f"\n{top_k} nodes with lowest farness values:")
    # for node, farness in k_lowest_farness:
    # print(f"Node: {node}, Farness: {farness}")

    Farn = initial_lower_bounds.copy()
    Top = []
    Q = [(Farn[node], node) for node in graph.nodes()]
    heapq.heapify(Q)

    initial_threshold = float('inf')

    _, v = heapq.heappop(Q)

    # print(f"Processing node {v}")

    threshold = initial_threshold if len(Top) < top_k else Farn[Top[-1]]
    refined_bound = updateBoundsBFSCut(v, graph, threshold)

    # print(f"Refined bound for node {v}: {refined_bound}")

    Farn[v] = refined_bound

    while Q:
        if len(Top) < top_k:
            Top.append(v)
            Top.sort(key=lambda node: Farn[node])
            # print(f"Added node {v} to Top. Current Top: {Top}")
        elif refined_bound < Farn[Top[-1]]:
            Top.append(v)
            Top.sort(key=lambda node: Farn[node])
            Top = Top[:top_k]  # Maintain Top's length to top_k
            # print(f"Added node {v} to Top and trimmed. Current Top: {Top}")
        # else:
        # print(f"Node {v} not added to Top as its bound is not better than current top")

        _, v = heapq.heappop(Q)

        # print(f"Processing node {v}")

        threshold = initial_threshold if len(Top) < top_k else Farn[Top[-1]]
        refined_bound = updateBoundsBFSCut(v, graph, threshold)

        # print(f"Refined bound for node {v}: {refined_bound}")

        Farn[v] = refined_bound

        # if len(Top) == top_k and Farn[v] >= Farn[Top[-1]]:
        # print(Farn[Q[0][1]])
        # print(Farn[Top[-1]])
        # print("Remaining nodes in Q cannot improve Top. Stopping.")
        # break

    print("\nFinal Top 10 nodes by refined closeness centrality:")
    for i, node in enumerate(Top, 1):
        closeness = 1 / Farn[node] if Farn[node] != 0 else float('inf')
        print(f"{i}. Node: {node}, Refined Closeness Centrality pagerank: {closeness:.10f}")

    end_time = time.time()
    total_time = end_time - start_time
    print(f"\nTotal time taken: {total_time:.4f} seconds")

    return Top


def evaluate_methods(graph):

    graph = graph

    start_time = time.time()
    nx_top_k_nodes = calculate_top_k_closeness_centrality_nx(graph, top_k=top_k)
    time_nx_trials.append(time.time() - start_time)

    start_time = time.time()
    paper_top_k_nodes = calculate_top_k_closeness_centrality_paper(graph, top_k=top_k)
    time_paper_trials.append(time.time() - start_time)

    start_time = time.time()
    pagerank_top_k_nodes = calculate_top_k_closeness_centrality_pagerank(graph, top_k=top_k)
    time_pagerank_trials.append(time.time() - start_time)

    acc_paper_trials.append(len(set(paper_top_k_nodes) & set(nx_top_k_nodes)) / top_k)
    acc_pagerank_trials.append(len(set(pagerank_top_k_nodes) & set(nx_top_k_nodes)) / top_k)

    common_nodes_nx_paper = set(paper_top_k_nodes) & set(nx_top_k_nodes)
    print(f"\nNumber of common nodes in paper and nx methods: {len(common_nodes_nx_paper)}")
    print(f"Common nodes: {common_nodes_nx_paper}")

    common_nodes_nx_pagerank = set(pagerank_top_k_nodes) & set(nx_top_k_nodes)
    print(f"\nNumber of common nodes in pagerank and nx methods: {len(common_nodes_nx_pagerank)}")
    print(f"Common nodes: {common_nodes_nx_pagerank}")

    print(f"Completed evaluation for graph with {num_nodes} nodes and {num_edges} edges")

    return accuracy_paper, accuracy_pagerank, time_nx, time_paper, time_pagerank


def plot_results(num_nodes_list, accuracy_paper, accuracy_pagerank, time_nx, time_paper, time_pagerank):
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(num_nodes_list, accuracy_paper, label='Paper Method')
    plt.plot(num_nodes_list, accuracy_pagerank, label='PageRank Method')
    plt.xlabel('Number of Nodes')
    plt.ylabel('Accuracy')
    plt.title('Accuracy Comparison')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(num_nodes_list, time_nx, label='NetworkX')
    plt.plot(num_nodes_list, time_paper, label='Paper Method')
    plt.plot(num_nodes_list, time_pagerank, label='PageRank Method')
    plt.xlabel('Number of Nodes')
    plt.ylabel('Time (seconds)')
    plt.title('Time Comparison')
    plt.legend()

    plt.tight_layout()
    plt.show()


def load_graph_from_tsv(file_path):
    df = pd.read_csv(file_path, sep='\t', header=None, names=['source', 'target'])

    graph = nx.from_pandas_edgelist(df, 'source', 'target')

    print(f"Graph loaded with {graph.number_of_nodes()} nodes and {graph.number_of_edges()} edges.")
    return graph


if __name__ == "__main__":
    file_path = 'com-youtube.ungraph.tsv'
    graph = load_graph_from_tsv(file_path)

    accuracy_paper, accuracy_pagerank, time_nx, time_paper, time_pagerank = evaluate_methods(num_nodes_list
    print(f'\nNumber of common nodes in pagerank and nx methods: {len(common_nodes_nx_pagerank)}')